# WLC Rogue List
This notebook polls a Cisco Wireless LAN Controller via SNMPv2 and returns a list of rogue AP's seen by that WLC.  

Follow the notebook, running each cell in order by highlighting the cell and pressing Shift-Enter.  

## Load Modules

In [ ]:
import pysnmp
from pysnmp.hlapi import *
import re
import pandas as pd

def snmp_v2_walk(host_name, community_name, oid):
    for (errorIndication,
         errorStatus,
         errorIndex,
         varBinds) in nextCmd(SnmpEngine(), 
                              CommunityData(community_name),
                              UdpTransportTarget((host_name, 161)),
                              ContextData(),                                                           
                              ObjectType(ObjectIdentity(oid)),
                              lexicographicMode=False):
        if errorIndication:
            print(errorIndication)
        elif errorStatus:
            print('%s at %s' % (errorStatus.prettyPrint(),
                                errorIndex and varBinds[int(errorIndex) - 1][0] or '?'))
        else:
            for varBind in varBinds:
                print(' = '.join([x.prettyPrint() for x in varBind]))

                

def snmp_v2_walk_iter(host_name, community_name, oid):
    for (errorIndication,
         errorStatus,
         errorIndex,
         varBinds) in nextCmd(SnmpEngine(), 
                              CommunityData(community_name),
                              UdpTransportTarget((host_name, 161)),
                              ContextData(),                                                           
                              ObjectType(ObjectIdentity(oid)),
                              lexicographicMode=False):
        if errorIndication:
            print(errorIndication)
        elif errorStatus:
            print('%s at %s' % (errorStatus.prettyPrint(),
                                errorIndex and varBinds[int(errorIndex) - 1][0] or '?'))
        else:
            for varBind in varBinds:
                out = ' = '.join([x.prettyPrint() for x in varBind])
        yield(out)



def snmp_v2_get(host_name, community_name, oid):

    errorIndication, errorStatus, errorIndex, varBinds = next(
        getCmd(SnmpEngine(),
               CommunityData(community_name),
               UdpTransportTarget((host_name, 161)),
               ContextData(),
               ObjectType(ObjectIdentity(oid)))
    )

    if errorIndication:
        print(errorIndication)
    elif errorStatus:
        print('%s at %s' % (errorStatus.prettyPrint(),
                            errorIndex and varBinds[int(errorIndex) - 1][0] or '?'))
    else:
        for varBind in varBinds:
            print(' = '.join([x.prettyPrint() for x in varBind]))
            

## Set SNMP Parameters
Add your own information here


In [ ]:
wlc_ip_address = <IP ADDRESS>
snmp_v2_community = <COMMUNITY STRING>


## Put all Rogue Data in DataFrame

In [ ]:
oid_rogue = '1.3.6.1.4.1.9.9.610.1.1.8.1.1'
pattern = re.compile("([0-9]+)\.(.*)")

df = pd.DataFrame(columns=['ID', 'key', 'value'])

for line in snmp_v2_walk_iter(wlc_ip_address, snmp_v2_community, oid_rogue):

    key, value = line.split(' = ')
    key2 = key.split('.9.9.610.1.1.8.1.1.')[1]

    match = re.split(pattern=pattern, string=key2)
    value_key = match[1]
    value_id = match[2]
    val_dict = {'ID':[value_id], 'key':[value_key], 'value':[value]}
    df = df.append(pd.DataFrame.from_dict(val_dict))
  

## Format into Database

In [ ]:
id_list = df['ID'].unique()

df_output = pd.DataFrame()

for rogue_id in id_list:
    f1 = df['ID'] == rogue_id
    f2 = df['key'] == '1'
    try:
        rogue_bssid = df.loc[f1&f2, 'value'][0]
    except:
        rogue_bssid = "N/A"
    f2 = df['key'] == '4'
    try:
        rogue_ap_name = df.loc[f1&f2, 'value'][0]
    except:
        rogue_ap_name = "N/A"
    f2 = df['key'] == '5'
    try:
        rogue_channel = df.loc[f1&f2, 'value'][0]
    except:
        rogue_channel = "N/A"
    f2 = df['key'] == '6'
    try:
        rogue_ssid = df.loc[f1&f2, 'value'][0]
    except:
        rogue_ssid = "N/A"
    f2 = df['key'] == '8'
    try:
        rogue_rssi = df.loc[f1&f2, 'value'][0]
    except:
        rogue_rssi = "N/A"
    f2 = df['key'] == '13'
    try:
        rogue_security_wpa = df.loc[f1&f2, 'value'][0]
    except:
        rogue_security_wpa = "N/A"
    df_output.loc[rogue_id, 'BSSID'] = rogue_bssid
    df_output.loc[rogue_id, 'AP Name'] = rogue_ap_name
    df_output.loc[rogue_id, 'Channel'] = rogue_channel
    df_output.loc[rogue_id, 'SSID'] = rogue_ssid
    df_output.loc[rogue_id, 'RSSI'] = rogue_rssi
    df_output.loc[rogue_id, 'WPA'] = rogue_security_wpa

    
df_output

## Save Data to Local CSV File

In [ ]:
df_output.to_csv("rogue_data.csv")

## Filter Data for Unsecure Rogue SSID's

In [ ]:
f1 = df_output['WPA'] == '0'
df_output.loc[f1, :]